In [2]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output

Game = Regency()
clear_output()
for a in range(1):
    Game.random_events()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))

Game.clear_screen()
Game.Seasons[Game.Season]['Season']

SyntaxError: invalid syntax (regency.py, line 3536)

In [ ]:
Game.get_travel_cost('JR', 'Bogsend', 'Sunken Lands', 'Archers')

In [ ]:
self = Game
Regent = 'EC'
import pandas as pd
import numpy as np

temp = pd.merge(self.Provences[self.Provences['Regent']=='EC'][['Regent', 'Provence']], self.Troops, on='Provence', how='left').fillna(0)
temp = temp[temp['Type'] != 0]
temp = temp[['Provence', 'CR']].groupby('Provence').sum().reset_index()
temp['roll'] = np.random.randint(1, 100)+temp['CR']
temp = temp.sort_values('roll', ascending=False)

temp

In [ ]:
Game.Troops

In [ ]:
temp = self.Troops[self.Troops['Regent']=='EC'].copy()
temp = temp[temp['Type'] == 'Pikemen']
temp = temp[temp['Provence'] == 'Bogsend']

temp

In [ ]:
for i, row in temp.iterrows():
    print(i, row)

In [ ]:
self = Game
Regent = 'EC'

race = self.Regents[self.Regents['Regent']==Regent]['Race'].values[0]
temp = pd.merge(self.Holdings[self.Holdings['Regent']==Regent].copy()
                , self.troop_units[self.troop_units['Type'] == race].copy()
                , left_on=['Type', 'Level'], right_on=['Requirements Holdings', 'Requirements Level']
                , how='left')
temp = temp[temp['Unit Type'] != 'Levies']
# can I afford it
gold = self.Regents[self.Regents['Regent'] == Regent]['Gold Bars'].values[0]
temp = temp[temp['Muster Cost'] <= gold]

temp.iloc[0]['Unit Type']

In [ ]:
self.Troops

In [ ]:
temp

In [ ]:
Regent = 'LPA'
Type = 'Irregulars'
Provence = 'Sunken Lands'
Garrisoned = 0
Home = ''
self.Troops = self.Troops.append(pd.DataFrame([[Regent
														   , Provence
														   , Type
														   , self.troop_units[self.troop_units['Unit Type'] == Type]['Maintenance Cost'].values[0]
														   , self.troop_units[self.troop_units['Unit Type'] == Type]['BCR'].values[0]
														   , Garrisoned
														   , Home]], columns=['Regent', 'Provence', 'Type', 'Cost', 'CR', 'Garrisoned', 'Home']))

In [ ]:
self.Troops

In [ ]:
self.Troops = self.Troops.append(pd.DataFrame([[Regent
														   , Provence
														   , Type
														   , self.troop_units[self.troop_units['Unit Type'] == Type]['Maintenance Cost'].values[0]
														   , self.troop_units[self.troop_units['Unit Type'] == Type]['BCR'].values[0]
														   , Garrisoned
														   , Home]], columns=['Regent', 'Provence', 'Type', 'Cost', 'CR', 'Garrisoned', 'Home'])
                                                           , ignore_index=True)

In [ ]:
self.Troops